In [1]:
import cv2

video_path = "data/session5_center/video.avi"
json_gtruth_path = 'data/session5_center_gtruth/system_dubska_bmvc14.json'

In [2]:
import pickle

with open('data/session5_center/gt_data.pkl', 'rb') as pkl_file:
    speed_data = pickle.load(pkl_file, encoding='latin')

In [3]:
speed_data.keys()

dict_keys(['distanceMeasurement', 'measurementLines', 'cars', 'fps', 'invalidLanes', 'laneDivLines'])

In [4]:
speed_data['measurementLines']

[array([-3.84424698e-02,  9.99260815e-01, -9.06560317e+02]),
 array([-1.23795715e-03,  9.99999234e-01, -8.48335447e+01])]

In [5]:
speed_data['cars']

[{'acceleration': -1.299,
  'intersections': [{'measurementLineId': 1, 'videoTime': 34.835276},
   {'measurementLineId': 0, 'videoTime': 36.147998}],
  'carId': 1,
  'timeIntersectionLastShifted': 36.147998,
  'valid': True,
  'laneIndex': {0},
  'speed': 76.8348},
 {'acceleration': 0.124,
  'intersections': [{'measurementLineId': 1, 'videoTime': 38.892936},
   {'measurementLineId': 0, 'videoTime': 40.080034}],
  'carId': 2,
  'timeIntersectionLastShifted': 40.080034,
  'valid': True,
  'laneIndex': {0},
  'speed': 84.9636},
 {'acceleration': 0.086,
  'intersections': [{'measurementLineId': 1, 'videoTime': 41.393706},
   {'measurementLineId': 0, 'videoTime': 42.693296000000004}],
  'carId': 3,
  'timeIntersectionLastShifted': 42.693296000000004,
  'valid': True,
  'laneIndex': {0},
  'speed': 77.6088},
 {'acceleration': 0.373,
  'intersections': [{'measurementLineId': 1, 'videoTime': 44.132212},
   {'measurementLineId': 0, 'videoTime': 45.420876}],
  'carId': 4,
  'timeIntersectionLast

In [7]:
speed_data['measurementLines']

[array([-3.84424698e-02,  9.99260815e-01, -9.06560317e+02]),
 array([-1.23795715e-03,  9.99999234e-01, -8.48335447e+01])]

In [8]:
import json

with open(json_gtruth_path, 'r') as json_file:
    position_data = json.load(json_file)

cars_data = position_data['cars']

In [22]:
import math

def play_speed_vid(video_path, gtruth, set_time_seconds = 34):
    
    speeds = gtruth['cars']
    speed_iter = iter(speeds)
    carspeed_dict = next(speed_iter)
    
    while not carspeed_dict['valid'] or carspeed_dict['intersections'][0]['videoTime'] < set_time_seconds:
        carspeed_dict = next(speed_iter)
            
    cap = cv2.VideoCapture(video_path)
        
    cap.set(cv2.CAP_PROP_POS_MSEC,set_time_seconds*1000) 
  
    while cap.isOpened():
        
        ret, frame = cap.read()
        
        if ret:
            
            current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
            current_time_trunc = float('{:.2f}'.format(current_time))
            
            text = f"Video time: {current_time_trunc}"
            cur_id_text = f"Id: {carspeed_dict['carId']}"
            
            cv2.putText(frame, text, (1500, 160), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, 2)
            cv2.putText(frame, cur_id_text, (1500, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, 2)
            
            time_first_line = carspeed_dict['intersections'][0]['videoTime']
            time_last_line = carspeed_dict['intersections'][1]['videoTime']
            
            if current_time >= time_first_line and current_time <= time_last_line:
                speed_txt = str(float("{:.2f}".format(carspeed_dict['speed']))) + " km/h"
                speed_txt_org = tuple()
                
                if carspeed_dict['laneIndex'] == {1}:
                    speed_txt_org = (1600, 840)
                else:
                    speed_txt_org = (450, 840)

                cv2.putText(frame, speed_txt, speed_txt_org, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, 2)
                
                if math.isclose(current_time, time_last_line, rel_tol=0.001):
                    carspeed_dict = next(speed_iter)
                    if not carspeed_dict['valid']:
                        carspeed_dict = next(speed_iter)    
            
            cv2.imshow('t', frame)
            
            key = cv2.waitKey(20)
                        
            if key & 0xFF == ord('q'):
                break
            if key & 0xFF == ord('p'):
                cv2.waitKey(-1)
            
        else:
            break
    
    
    cap.release()
    cv2.destroyAllWindows()
    
   

In [23]:
play_speed_vid(video_path, speed_data,50)

In [11]:
def get_frame_from_video_time(video_path, time):
    
        
    cap = cv2.VideoCapture(video_path)
        
    cap.set(cv2.CAP_PROP_POS_MSEC,time*1000) 
    
    _, frame = cap.read()
    
    cv2.imshow('s', frame)
    
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cap.release()

In [14]:
get_frame_from_video_time(video_path, 40.080034)